In [3]:
from flask import Flask, request, send_from_directory
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.sql.expression import func
import random
import re
import requests
import os

# 定义静态文件夹路径
ASSETS_FOLDER = os.path.join(os.getcwd(), "assets")


def process_description(description):
    # 匹配所有 <img src="URL" ... />
    img_tags = re.findall(r'<img src="https?://[^"]+"[^>]*>', description)
    # 移除所有 <img src="URL" ... />
    description_without_imgs = re.sub(
        r'<img src="https?://[^"]+"[^>]*>', "", description
    )
    # 如果有多个 <img src="URL" ... />，只保留最后一个
    if img_tags:
        description_updated = description_without_imgs.strip() + " " + img_tags[-1]
    else:
        description_updated = description_without_imgs.strip()
    return description_updated


def download_image(url, folder_path):
    # 获取图片的名字
    image_name = url.split("/")[-1]
    # 清理文件名，移除潜在的目录遍历字符
    image_name = os.path.basename(image_name)
    # 防止路径注入，确保文件名不包含目录路径
    if ".." in image_name or "/" in image_name or "\\" in image_name:
        raise ValueError("无效的文件名")
    # 完整的文件路径
    file_path = os.path.join(folder_path, image_name)
    # 发送GET请求下载图片，忽略SSL证书验证
    response = requests.get(url, verify=False)
    # 检查请求是否成功
    if response.status_code == 200:
        # 打开文件并写入图片数据
        with open(file_path, "wb") as file:
            file.write(response.content)
        print(f"图片已下载到：{file_path}")
        # 返回本地服务的URL路径
        local_url = f"http://127.0.0.1:5000/assets/{image_name}"
        return local_url
    else:
        print(f"下载失败，URL: {url}")
        return None


def process_description_and_download_images(description, folder_path):
    # 匹配所有 <img src="URL" ... />，同时支持单引号和双引号
    img_tags = re.findall(r'<img src=([\'"])(https?://[^\'"]+)[\'"][^>]*>', description)
    downloaded_files = []  # 用于收集下载的文件路径
    # 下载所有匹配到的图片并替换描述中的URL为本地服务的URL
    for match in img_tags:
        quote_type, img_url = match
        local_url = download_image(img_url, folder_path)
        if local_url:  # 确保只处理成功下载的图片
            downloaded_files.append(local_url)
            # 替换描述中的图片URL为本地服务的URL
            description = description.replace(
                f"{quote_type}{img_url}{quote_type}",
                f"{quote_type}{local_url}{quote_type}",
            )
    # 移除所有 <img src="URL" ... />，并返回更新后的描述
    description_without_imgs = re.sub(
        r'<img src=[\'"]https?://[^\'"]+[\'"][^>]*>', "", description
    ).strip()
    return description_without_imgs, downloaded_files  # 返回更新后的描述和文件路径列表


app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///fomeow.db"  # 使用 SQLite 数据库
db = SQLAlchemy(app)

with open("pathdata.txt", "r") as file:
    file_paths = [line.strip() for line in file]


class Food(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    description = db.Column(db.String(120), nullable=False)
    count = db.Column(db.Integer, default=0)  # 新增的 count 属性


with app.app_context():
    db.create_all()


@app.route("/add", methods=["GET"])
def api():
    name = request.args.get("name")
    description = request.args.get("描述")
    food = Food.query.filter_by(name=name).first()
    if food is None:
        food = Food(name=name, description=description)
        db.session.add(food)
        message = f"已添加: 名称-{name}, 描述-{description}"
    else:
        food.description += " " + description
        message = f"已更新: 名称-{name}, 描述-{food.description}"
    db.session.commit()
    return {"message": message}


@app.route("/random", methods=["GET"])
def random_food():
    food = Food.query.order_by(func.random()).first()
    if food is None:
        return {"message": "数据库中没有食物"}
    else:
        if food.count is None:  # 如果 count 是 None，初始化为 0
            food.count = 0
        food.count += 1  # 随机到了这个食物，count 加一
        db.session.commit()
        return {"name": food.name, "description": food.description}


@app.route("/remove", methods=["GET"])
def remove_description():
    name = request.args.get("name")
    description_to_remove = request.args.get("描述")
    food = Food.query.filter_by(name=name).first()
    if food is None:
        return {"message": f"没有找到名称为 {name} 的食物"}
    else:
        if description_to_remove in food.description:
            food.description = food.description.replace(description_to_remove, "")
            db.session.commit()
            return {"message": f"已从 {name} 的描述中删除 {description_to_remove}"}
        else:
            return {"message": f"{name} 的描述中没有 {description_to_remove}"}


@app.route("/query", methods=["GET"])
def query_food():
    name = request.args.get("name")
    food = Food.query.filter_by(name=name).first()
    if food is None:
        return {"message": f"没有找到名称为 {name} 的食物"}
    else:
        return {
            "message": f"{food.name} 的描述是 {food.description}，被随机到的次数是 {food.count}"
        }


@app.route("/delete", methods=["GET"])
def delete_food():
    name = request.args.get("name")
    food = Food.query.filter_by(name=name).first()
    if food is None:
        return {"message": f"没有找到名称为 {name} 的食物"}
    else:
        db.session.delete(food)
        db.session.commit()
        return {"message": f"已删除名称为 {name} 的食物"}


@app.route("/food", methods=["GET"])
def send_file():
    # 从 file_paths 中随机选择一个文件路径
    file_path = random.choice(file_paths)
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
    return {"message": content}


@app.route("/config", methods=["GET"])
def config():
    name = request.args.get("name")
    # 使用正则表达式匹配 <img src="URL" ... />
    # img_urls = re.findall(r'<img src="(.*?)"', raw_description)
    food = Food.query.filter_by(name=name).first()
    # 将所有匹配到的 URL 以空格分隔
    # description = " ".join(img_urls)
    food = Food.query.filter_by(name=name).first()
    if food is None:
        # food = Food(name=name, description=description, count=0)
        # db.session.add(food)
        # message = f"已添加: 名称-{name}, 图片 URLs-{description}"
        return {"message": f"没有找到名称为 {name} 的食物"}
    else:
        food.description = process_description(food.description)
        message = f"已更新: 名称-{name}, 描述-{food.description}"
    db.session.commit()
    return {"message": message}


@app.route("/assets/<path:filename>")
def serve_assets(filename):
    # 从静态文件夹中发送请求的文件
    return send_from_directory(ASSETS_FOLDER, filename)


if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
